In [329]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [330]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [331]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [332]:
async def get_expiry(f_today, index):

    if index == 'bnf':    
        if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
            f_expiry = dt.date(2024, 1, 25)
        elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
            f_expiry = dt.date(2024, 1, 31)
        elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
            f_expiry = dt.date(2024, 2, 29)
        elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
            f_expiry = dt.date(2024, 2, 27)
        elif f_today < dt.date(2023, 9, 1):
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
        elif f_today >= dt.date(2023, 9, 1):
            if f_today.day < 24:
                days_to_wednesday = (2 - f_today.weekday()) % 7
                nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
                f_expiry = nearest_wednesday
                if nse.valid_days(
                    start_date=nearest_wednesday, end_date=nearest_wednesday
                ).empty:
                    f_expiry = nearest_wednesday - dt.timedelta(days=1)
            else:
                days_to_thursday = (3 - f_today.weekday()) % 7
                nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
                f_expiry = nearest_thursday
                if nse.valid_days(
                    start_date=nearest_thursday, end_date=nearest_thursday
                ).empty:
                    f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'nifty':
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'finnifty' or index == 'fnf':
        days_to_thursday = (1 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'midcpnifty' or index == 'midcp':
        days_to_thursday = (0 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

async def get_expiry_nifty(f_today):

    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"


def get_option_contract_name2(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [333]:
bnf_1min = pd.read_csv("../data/ng_4h.csv")
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["time"])
bnf_1min = bnf_1min[bnf_1min["datetime"].dt.year >= 2012]

In [334]:
bnf_1min.head()

,time,open,high,low,close,datetime
0,2015-03-02T09:00:00+05:30,169.0000,169.0000,167.8000,168.3000,2015-03-02 09:00:00+05:30
1,2015-03-02T13:00:00+05:30,168.3000,168.8000,167.8000,168.4000,2015-03-02 13:00:00+05:30
2,2015-03-02T17:00:00+05:30,168.3000,171.7000,168.2000,170.9000,2015-03-02 17:00:00+05:30
3,2015-03-02T21:00:00+05:30,170.9000,171.7000,167.9000,169.1000,2015-03-02 21:00:00+05:30
4,2015-03-03T09:00:00+05:30,168.8000,168.8000,166.1000,166.6000,2015-03-03 09:00:00+05:30


In [335]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [336]:
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"])
list_of_traded_dates = set(bnf_1min["datetime"].dt.date)
list_of_traded_dates

{datetime.date(2022, 1, 27),
 datetime.date(2017, 1, 20),
 datetime.date(2022, 8, 11),
 datetime.date(2023, 2, 24),
 datetime.date(2016, 10, 17),
 datetime.date(2018, 7, 13),
 datetime.date(2017, 5, 5),
 datetime.date(2020, 6, 9),
 datetime.date(2023, 3, 17),
 datetime.date(2015, 3, 6),
 datetime.date(2023, 5, 3),
 datetime.date(2022, 5, 10),
 datetime.date(2017, 2, 27),
 datetime.date(2015, 6, 8),
 datetime.date(2022, 8, 16),
 datetime.date(2018, 1, 19),
 datetime.date(2023, 5, 16),
 datetime.date(2020, 1, 9),
 datetime.date(2024, 9, 11),
 datetime.date(2015, 3, 18),
 datetime.date(2024, 8, 30),
 datetime.date(2016, 8, 4),
 datetime.date(2020, 11, 18),
 datetime.date(2023, 9, 6),
 datetime.date(2020, 8, 24),
 datetime.date(2015, 10, 29),
 datetime.date(2017, 2, 15),
 datetime.date(2020, 1, 24),
 datetime.date(2022, 1, 20),
 datetime.date(2020, 7, 29),
 datetime.date(2021, 11, 22),
 datetime.date(2020, 3, 11),
 datetime.date(2024, 1, 17),
 datetime.date(2020, 6, 23),
 datetime.date(201

In [337]:
def rename_ohlc_columns(df: pl.DataFrame) -> pl.DataFrame:

    column_mapping = {"o": "open", "h": "high", "l": "low", "c": "close", "v": "volume"}
    df = df.rename(column_mapping)

    return df

In [338]:
async def create_sell_signals(df):

    df["sell_signal"] = False

    df["sell_signal"] = (df["high"] < df["high"].shift(1)) & (
        df["high"].shift(2) < df["high"].shift(1)
    )

    return df

async def create_buy_signals(df):
    df["buy_signal"] = False

    df["buy_signal"] = (df["low"] > df["low"].shift(1)) & (
        df["low"].shift(2) > df["low"].shift(1)
    )

    return df

async def create_ma(df, period):
    df["close"] = pd.to_numeric(df["close"], errors="coerce")
    df["ma"] = df["close"].rolling(window=period).mean()
    return df

In [366]:
PORTFOLIO_VALUE = 10_00_000 # 10 Lacs
RPT_PCT = 0.01 # 1% RPT
SLIPPAGE_ = 0.001
LEVERAGE_ = 5

In [367]:
def positional(df, sl_pct, target_pct):
    trades = []
    in_trade_long = False
    in_trade_short = False
    current_trade = None
    king_candle = None
    queen_candle = None
    slippage_pct = SLIPPAGE_
    portfolio = 10000000
    index_lev = 6
    entry_time = None
    exit_time = None
    is_trailing_active_long = False
    is_trailing_active_short = False
    points_long = None
    points_short = None
    initial_sl_long = 100000
    initial_sl_short = 0

    df['100ma'] = df['close'].rolling(100).mean()

    for i in range(2, len(df)):
        prev2_candle = df.iloc[i-2]
        prev_candle = df.iloc[i-1]
        current_candle = df.iloc[i]

        # Check for King candle formation when not in a trade
        if not in_trade_long and current_candle['high'] < prev_candle['high'] and prev2_candle['high'] < prev_candle['high'] and prev_candle['high'] < prev_candle['100ma']:
            king_candle = prev_candle
            signal_time_long = king_candle['datetime']
            initial_sl_long = min(current_candle['low'], prev_candle['low'], prev2_candle['low'])

        # Check for King breakout (Long position)
        if king_candle is not None and not in_trade_long and current_candle['high'] > king_candle['high']:
            # if (current_candle['open'] > king_candle['high'] and current_candle['low'] <= king_candle['high']) or (current_candle['open'] <= king_candle['high']):
            #     entry_price_long = king_candle['high']
            #     stop_loss_long = entry_price_long * (1 - sl_pct)
            #     # stop_loss_long = initial_sl_long
            #     target_long = entry_price_long * (1 + target_pct)
            #     entry_time_long = df.iloc[i]['datetime']
            #     in_trade_long = True
            #     # print(f"Taking Long Position: Entry={entry_price}, SL={stop_loss}, Target={target}")
            # else:
            #     entry_price_long = current_candle['close']
            #     stop_loss_long = entry_price_long * (1 - sl_pct)
            #     # stop_loss_long = initial_sl_long
            #     target_long = entry_price_long * (1 + target_pct)
            #     entry_time_long = df.iloc[i]['datetime']
            #     in_trade_long = True
            #     # print(f"Taking Long Position: Entry={entry_price}, SL={stop_loss}, Target={target}")
            if (current_candle['open'] > king_candle['high'] and current_candle['low'] <= king_candle['high']) or (current_candle['open'] <= king_candle['high']):
                entry_price_long = king_candle['high']
                # stop_loss_long = entry_price_long * (1 - sl_pct)
                stop_loss_long = initial_sl_long
                target_long = entry_price_long * (1 + target_pct)
                entry_time_long = df.iloc[i]['datetime']
                in_trade_long = True
            #     # print(f"Taking Long Position: Entry={entry_price}, SL={stop_loss}, Target={target}")
            elif current_candle['open'] > king_candle['high']:
                king_candle = None
                in_trade_long = False

        if in_trade_long:
            if not is_trailing_active_long:
                if current_candle['low'] > current_candle['ma']:
                    is_trailing_active_long = True
            if not is_trailing_active_long:
                if (current_candle['low'] <= stop_loss_long):
                    if current_candle['open'] < stop_loss_long:
                        if current_candle['datetime'] != entry_time_long:
                            exit_price_long = current_candle['open']
                            exit_time_long = df.iloc[i]['datetime']
                            points_long = exit_price_long - entry_price_long
                            remarks_long = 'Gap SL'
                    else:
                        exit_price_long = stop_loss_long
                        exit_time_long = df.iloc[i]['datetime']
                        points_long = exit_price_long - entry_price_long
                        remarks_long = 'Initial SL Hit'
                    
                elif (current_candle['high'] >= target_long):
                    exit_price_long = target_long
                    exit_time_long = df.iloc[i]['datetime']
                    points_long = exit_price_long - entry_price_long
                    remarks_long = 'Target Hit'
            else:
                if (current_candle['low'] <= stop_loss_long):
                    if current_candle['open'] < stop_loss_long:
                        if current_candle['datetime'] != entry_time_long:
                            exit_price_long = current_candle['open']
                            exit_time_long = df.iloc[i]['datetime']
                            points_long = exit_price_long - entry_price_long
                            remarks_long = 'Gap SL'
                    else:
                        exit_price_long = stop_loss_long
                        exit_time_long = df.iloc[i]['datetime']
                        points_long = exit_price_long - entry_price_long
                        remarks_long = 'Initial SL Hit'
                    
                elif (current_candle['high'] >= target_long):
                    exit_price_long = target_long
                    exit_time_long = df.iloc[i]['datetime']
                    points_long = exit_price_long - entry_price_long
                    remarks_long = 'Target Hit'

                elif (current_candle['close'] <= current_candle['ma']):
                    exit_price_long = current_candle['close']
                    exit_time_long = df.iloc[i]['datetime']
                    points_long = exit_price_long - entry_price_long
                    remarks_long = 'Trailing SL Hit'

            if points_long:
                slippages = SLIPPAGE_ * (entry_price_long + exit_price_long)
                # qty = portfolio * index_lev / entry_price_long
                qty = portfolio * RPT_PCT / abs(entry_price_long - initial_sl_long)
                current_trade = {
                    'Type': 'LONG',
                    'Signal Generated At': signal_time_long,
                    'Entry Time': entry_time_long,
                    'Entry Price': entry_price_long,
                    'Initial SL': stop_loss_long,
                    'Initial Target': target_long,
                    'Exit Time': exit_time_long,
                    'Exit Price': exit_price_long,
                    'Points': points_long,
                    'Slippages': slippages,
                    'Remarks': remarks_long,
                    'Qty': qty,
                    'PnL': points_long * qty,
                    'PnL w cs': (points_long - slippages) * qty,
                    'ROI%': points_long * qty * 100 / portfolio,
                    'ROI%': (points_long - slippages) * qty * 100 / portfolio,
                }
                trades.append(current_trade) 
                in_trade_long = False  
                king_candle = None  
                is_trailing_active_long = False
                points_long = None
                signal_time_long = None
                initial_sl_long = 100000

        # Check for Queen candle formation when not in a trade
        if not in_trade_short and current_candle['low'] > prev_candle['low'] and prev2_candle['low'] > prev_candle['low'] and prev_candle['low'] > prev_candle['100ma']:
            queen_candle = prev_candle
            signal_time_short = queen_candle['datetime']
            initial_sl_short = max(current_candle['high'], prev_candle['high'], prev2_candle['high'])
            
        # Check for Queen breakout (Short position)
        if queen_candle is not None and not in_trade_short and current_candle['low'] < queen_candle['low']:
            # if (current_candle['open'] < queen_candle['low'] and current_candle['high'] >= queen_candle['low']) or (current_candle['open'] >= queen_candle['low']):
            #     entry_price_short = queen_candle['low']
            #     stop_loss_short = entry_price_short * (1 + sl_pct)
            #     # stop_loss_short = initial_sl_short
            #     target_short = entry_price_short * (1 - target_pct)
            #     entry_time_short = df.iloc[i]['datetime']
            #     in_trade_short = True
            # else:
            #     entry_price_short = current_candle['close']
            #     stop_loss_short = entry_price_short * (1 + sl_pct)
            #     # stop_loss_short = initial_sl_short
            #     target_short = entry_price_short * (1 - target_pct)
            #     entry_time_short = df.iloc[i]['datetime']
            #     in_trade_short = True
            if (current_candle['open'] < queen_candle['low'] and current_candle['high'] >= queen_candle['low']) or (current_candle['open'] >= queen_candle['low']):
                entry_price_short = queen_candle['low']
                # stop_loss_short = entry_price_short * (1 + sl_pct)
                stop_loss_short = initial_sl_short
                target_short = entry_price_short * (1 - target_pct)
                entry_time_short = df.iloc[i]['datetime']
                in_trade_short = True
            elif current_candle['open'] < queen_candle['low']:
                queen_candle = None
                in_trade_short = False
            
        if in_trade_short:
            if not is_trailing_active_short:
                if current_candle['high'] < current_candle['ma']:
                    is_trailing_active_short = True
            if not is_trailing_active_short:
                if (current_candle['high'] >= stop_loss_short):
                    if current_candle['open'] > stop_loss_short:
                        if current_candle['datetime'] != entry_time_short:
                            exit_price_short = current_candle['open']
                            exit_time_short = df.iloc[i]['datetime']
                            points_short = entry_price_short - exit_price_short
                            remarks_short = 'Gap SL'
                    else:
                        exit_price_short = stop_loss_short
                        exit_time_short = df.iloc[i]['datetime']
                        points_short = entry_price_short - exit_price_short
                        remarks_short = 'Initial SL Hit'
                    
                elif (current_candle['low'] <= target_short):
                    exit_price_short = target_short
                    exit_time_short = df.iloc[i]['datetime']
                    points_short = entry_price_short - exit_price_short
                    remarks_short = 'Target Hit'
            else:
                if (current_candle['high'] >= stop_loss_short):
                    if current_candle['open'] > stop_loss_short:
                        if current_candle['datetime'] != entry_time_short:
                            exit_price_short = current_candle['open']
                            exit_time_short = df.iloc[i]['datetime']
                            points_short = entry_price_short - exit_price_short
                            remarks_short = 'Gap SL'
                    else:
                        exit_price_short = stop_loss_short
                        exit_time_short = df.iloc[i]['datetime']
                        points_short = entry_price_short - exit_price_short
                        remarks_short = 'Initial SL Hit'
                    
                elif (current_candle['low'] <= target_short):
                    exit_price_short = target_short
                    exit_time_short = df.iloc[i]['datetime']
                    points_short = entry_price_short - exit_price_short
                    remarks_short = 'Target Hit'

                elif (current_candle['close'] >= current_candle['ma']):
                    exit_price_short = current_candle['close']
                    exit_time_short = df.iloc[i]['datetime']
                    points_short = entry_price_short - exit_price_short
                    remarks_short = 'Trailing SL Hit'

            if points_short:
                slippages = SLIPPAGE_ * (entry_price_short + exit_price_short)
                # qty = portfolio * index_lev / entry_price_short
                qty = portfolio * RPT_PCT / abs(entry_price_short - initial_sl_short)
                current_trade = {
                    'Type': 'SHORT',
                    'Signal Generated At': signal_time_short,
                    'Entry Time': entry_time_short,
                    'Entry Price': entry_price_short,
                    'Initial SL': stop_loss_short,
                    'Initial Target': target_short,
                    'Exit Time': exit_time_short,
                    'Exit Price': exit_price_short,
                    'Points': points_short,
                    'Slippages': slippages,
                    'Remarks': remarks_short,
                    'Qty': qty,
                    'PnL': points_short * qty,
                    'PnL w cs': (points_short - slippages) * qty,
                    'ROI%': points_short * qty * 100 / portfolio,
                    'ROI%': (points_short - slippages) * qty * 100 / portfolio,
                }
                trades.append(current_trade)
                in_trade_short = False
                queen_candle = None
                is_trailing_active_short = False
                points_short = None
                signal_time_short = None
                initial_sl_short = 0
            
    return trades


# Example usage:
# trades = identify_trades(df)
# print(trades)


In [1]:
tf = '60m'
ofs = '15m'
ma_period = 50

# data = resample(pl.DataFrame(bnf_1min), tf, ofs)
# data_pandas = data.to_pandas()
data_pandas = bnf_1min
data_pandas['ma'] = data_pandas['close'].rolling(ma_period).mean()
# data['ma'] = data['close'].rolling(ma_period).mean()
tb = positional(data_pandas, 0.05, 0.25)

NameError: name 'bnf_1min' is not defined

In [369]:
tb_pandas = pd.DataFrame(tb)

In [370]:
tb_pandas['Entry Time'] = pd.to_datetime(tb_pandas['Entry Time'])
tb_pandas['Trade Year'] = tb_pandas['Entry Time'].dt.year
tb_pandas['ROI% w cs'] = tb_pandas['ROI%']

In [371]:
# (tb_pandas['pnl'].sum()) - (tb_pandas['slippage'].sum())
# tb_pandas = tb_pandas[tb_pandas['Type'] == 'LONG']
# tb_pandas

In [372]:
tb_pandas

,Type,Signal Generated At,Entry Time,Entry Price,Initial SL,Initial Target,Exit Time,Exit Price,Points,Slippages,Remarks,Qty,PnL,PnL w cs,ROI%,Trade Year,ROI% w cs
0,LONG,2015-04-13 17:00:00+05:30,2015-04-14 17:00:00+05:30,159.4000,155.8000,318.8000,2015-04-20 09:00:00+05:30,162.4000,3.0000,0.3218,Trailing SL Hit,27777.7778,83333.3333,74394.4444,0.7439,2015,0.7439
1,LONG,2015-04-21 17:00:00+05:30,2015-04-22 13:00:00+05:30,164.0000,160.7000,328.0000,2015-04-23 17:00:00+05:30,160.7000,-3.3000,0.3247,Initial SL Hit,30303.0303,-100000.0000,-109839.3939,-1.0984,2015,-1.0984
2,SHORT,2015-05-05 21:00:00+05:30,2015-05-06 13:00:00+05:30,176.5000,179.8000,0.0000,2015-05-06 17:00:00+05:30,179.8000,-3.3000,0.3563,Initial SL Hit,30303.0303,-100000.0000,-110796.9697,-1.1080,2015,-1.1080
3,SHORT,2015-05-07 09:00:00+05:30,2015-05-07 17:00:00+05:30,177.2000,180.7000,0.0000,2015-05-07 17:00:00+05:30,180.7000,-3.5000,0.3579,Initial SL Hit,28571.4286,-100000.0000,-110225.7143,-1.1023,2015,-1.1023
4,LONG,2015-04-28 17:00:00+05:30,2015-04-29 17:00:00+05:30,162.2000,158.7000,324.4000,2015-05-22 17:00:00+05:30,185.6000,23.4000,0.3478,Trailing SL Hit,28571.4286,668571.4286,658634.2857,6.5863,2015,6.5863
5,LONG,2015-06-01 13:00:00+05:30,2015-06-01 21:00:00+05:30,170.5000,167.6000,341.0000,2015-06-02 17:00:00+05:30,167.6000,-2.9000,0.3381,Initial SL Hit,34482.7586,-100000.0000,-111658.6207,-1.1166,2015,-1.1166
6,SHORT,2015-05-18 13:00:00+05:30,2015-05-19 21:00:00+05:30,189.9000,194.5000,0.0000,2015-06-09 13:00:00+05:30,176.4000,13.5000,0.3663,Trailing SL Hit,21739.1304,293478.2609,285515.2174,2.8552,2015,2.8552
7,LONG,2015-06-05 13:00:00+05:30,2015-06-08 09:00:00+05:30,169.8000,166.4000,339.6000,2015-06-18 21:00:00+05:30,177.4000,7.6000,0.3472,Trailing SL Hit,29411.7647,223529.4118,213317.6471,2.1332,2015,2.1332
8,SHORT,2015-06-16 17:00:00+05:30,2015-06-18 09:00:00+05:30,182.5000,188.6000,0.0000,2015-06-19 17:00:00+05:30,178.4000,4.1000,0.3609,Trailing SL Hit,16393.4426,67213.1148,61296.7213,0.6130,2015,0.6130
9,LONG,2015-06-24 17:00:00+05:30,2015-06-25 13:00:00+05:30,176.9000,172.7000,353.8000,2015-06-26 21:00:00+05:30,178.1000,1.2000,0.3550,Trailing SL Hit,23809.5238,28571.4286,20119.0476,0.2012,2015,0.2012


In [373]:
# tb_pandas.to_csv('Check KQ.csv', index=False)

In [374]:
# async def positional(df):

#     in_trade = False
#     tradebook = []
#     remark = ''
#     points = 0
#     entry_price = None
#     exit_price = None
#     initial_sl = None
#     # stoploss = None
#     is_trailing_active = False

#     for i in range(3, len(df)):
#         # SHORT Variation
    
#         if (not in_trade) and (df.iloc[i]['low'] < df.iloc[i-1]['low']) and (df.iloc[i-1]['sell_signal'] == True):
#             #Entry Triggered
#             if df.iloc[i]['open'] < df.iloc[i-1]['low']:
#                 if df.iloc[i]['high'] >= df.iloc[i-1]['low']:
#                     in_trade = True
#                     entry_price = df.iloc[i-1]['low']
#                     entry_time = df.iloc[i]['datetime']
#                     signal_time = df.iloc[i-1]['datetime']
#                     initial_sl = max(df.iloc[i-1]['high'], df.iloc[i-2]['high'], df.iloc[i-3]['high'])
#                     # stoploss = max(df.iloc[i-1]['high'], df.iloc[i-2]['high'], df.iloc[i-3]['high'])

#         if in_trade:
#             trade_entry_price = entry_price
#             trade_initial_sl = initial_sl
#             trade_final_sl = initial_sl

#             if (
#                 not is_trailing_active
#                 and df.iloc[i]['high'] < df.iloc[i]['ma']
#             ):
#                 is_trailing_active = True
            
#             if not is_trailing_active:
#                 if df.iloc[i]['open'] > trade_initial_sl:
#                     if (
#                         df.iloc[i]["datetime"].date() == entry_time.date()
#                         and df.iloc[i]["datetime"].time() == entry_time.time()
#                     ):
#                         if df.iloc[i]['close'] >= trade_initial_sl:
#                             in_trade = False
#                             points = trade_entry_price - trade_initial_sl
#                             exit_price = trade_initial_sl
#                             exit_time = df.iloc[i]["datetime"]
#                             remark = "Initial SL hit"

#                     else:
#                         # Gap Open Outside ISL
                        
#                         in_trade = False
#                         points = trade_entry_price - df.iloc[i]['open']
#                         exit_price = df.iloc[i]['open']
#                         exit_time = df.iloc[i]["datetime"]
#                         remark = "Gap Open Outside ISL"

#                 elif df.iloc[i]['high'] >= trade_initial_sl:
#                     # Initial SL Hit

#                     in_trade = False
#                     points = trade_entry_price - trade_initial_sl
#                     exit_price = trade_initial_sl
#                     exit_time = df.iloc[i]["datetime"]
#                     remark = "Initial SL Hit"

#             else:
#                 trade_final_sl = min(trade_initial_sl, df.iloc[i]['ma'])
#                 if df.iloc[i]['open'] > trade_initial_sl:
#                     if (
#                         df.iloc[i]["datetime"].date() == entry_time.date()
#                         and df.iloc[i]["datetime"].time() == entry_time.time()
#                     ):
#                         if df.iloc[i]['close'] >= trade_initial_sl:
#                             in_trade = False
#                             points = trade_entry_price - trade_initial_sl
#                             exit_price = trade_initial_sl
#                             exit_time = df.iloc[i]["datetime"]
#                             remark = "Initial SL hit"

#                     else:
#                         # Gap Open Outside ISL
                        
#                         in_trade = False
#                         points = trade_entry_price - df.iloc[i]['open']
#                         exit_price = df.iloc[i]['open']
#                         exit_time = df.iloc[i]["datetime"]
#                         remark = "Gap Open Outside ISL"

#                 elif df.iloc[i]['high'] >= trade_initial_sl:
#                     # Initial SL Hit

#                     in_trade = False
#                     points = trade_entry_price - trade_initial_sl
#                     exit_price = trade_initial_sl
#                     exit_time = df.iloc[i]["datetime"]
#                     remark = "Initial SL Hit"

#                 elif df.iloc[i]['close'] >= trade_final_sl:
#                     # Initial SL Hit

#                     in_trade = False
#                     points = trade_entry_price - df.iloc[i]['close']
#                     exit_price = df.iloc[i]['close']
#                     exit_time = df.iloc[i]["datetime"]
#                     remark = "TSL Hit"
    
#         if points:
#             slippage = SLIPPAGE_ * (entry_price + exit_price)
#             final_points = points - slippage
#             # sl_in_points = abs(entry_price - initial_sl)
#             qty = PORTFOLIO_VALUE * LEVERAGE_ / entry_price
#             trade = {
#                 'Signal Time': signal_time,
#                 'Trade Type': 'SHORT',
#                 'Entry Date': entry_time.date(),
#                 'Entry Time': entry_time.time(),
#                 'Entry Price': entry_price,
#                 'Initial SL': trade_initial_sl,
#                 'Final SL': trade_final_sl,
#                 'Exit Date': exit_time.date(),
#                 'Exit Time': exit_time.time(),
#                 'Exit Price': exit_price,
#                 'Points Captured': points,
#                 'After Costs': final_points,
#                 'Remarks': remark,
#                 'Qty': qty,
#                 'ROI%': (points * qty / PORTFOLIO_VALUE) * 100,
#                 'ROI% w cs': (final_points * qty / PORTFOLIO_VALUE) * 100,
#                 'Trade Year': entry_time.year,
#             }
#             # print(trade)
#             tradebook.append(trade)
#             points = 0
#             in_trade = False
#             remark = ''
#             entry_price = None
#             exit_price = None
#             initial_sl = None
#             # stoploss = None
#             is_trailing_active = False

#     for i in range(3, len(df)):
#         # LONG Variation
        
#         if (not in_trade) and (df.iloc[i]['high'] >= df.iloc[i-1]['high']) and (df.iloc[i-1]['buy_signal'] == True):
#             #Entry Triggered
#             if df.iloc[i]['open'] > df.iloc[i-1]['high']:
#                 if df.iloc[i]['low'] <= df.iloc[i-1]['high']:
#                     in_trade = True
#                     entry_price = df.iloc[i-1]['high']
#                     entry_time = df.iloc[i]['datetime']
#                     signal_time = df.iloc[i-1]['datetime']
#                     initial_sl = min(df.iloc[i-1]['low'], df.iloc[i-2]['low'], df.iloc[i-3]['low'])
#                     # stoploss = max(df.iloc[i-1]['low'], df.iloc[i-2]['low'], df.iloc[i-3]['low'])

#         if in_trade:
#             trade_entry_price = entry_price
#             trade_initial_sl = initial_sl
#             trade_final_sl = initial_sl

#             if (
#                 not is_trailing_active
#                 and df.iloc[i]['low'] > df.iloc[i]['ma']
#             ):
#                 is_trailing_active = True
            
#             if not is_trailing_active:
#                 if df.iloc[i]['open'] < trade_initial_sl:
#                     if (
#                         df.iloc[i]["datetime"].date() == entry_time.date()
#                         and df.iloc[i]["datetime"].time() == entry_time.time()
#                     ):
#                         if df.iloc[i]['close'] <= trade_initial_sl:
#                             in_trade = False
#                             points = trade_initial_sl - trade_entry_price
#                             exit_price = trade_initial_sl
#                             exit_time = df.iloc[i]["datetime"]
#                             remark = "Initial SL hit"

#                     else:
#                         # Gap Open Outside ISL
                        
#                         in_trade = False
#                         points = df.iloc[i]['open'] - trade_entry_price
#                         exit_price = df.iloc[i]['open']
#                         exit_time = df.iloc[i]["datetime"]
#                         remark = "Gap Open Outside ISL"

#                 elif df.iloc[i]['low'] <= trade_initial_sl:
#                     # Initial SL Hit

#                     in_trade = False
#                     points = trade_initial_sl - trade_entry_price
#                     exit_price = trade_initial_sl
#                     exit_time = df.iloc[i]["datetime"]
#                     remark = "Initial SL Hit"

#             else:
#                 trade_final_sl = max(trade_initial_sl, df.iloc[i]['ma'])
#                 if df.iloc[i]['open'] < trade_initial_sl:
#                     if (
#                         df.iloc[i]["datetime"].date() == entry_time.date()
#                         and df.iloc[i]["datetime"].time() == entry_time.time()
#                     ):
#                         if df.iloc[i]['close'] <= trade_initial_sl:
#                             in_trade = False
#                             points = trade_initial_sl - trade_entry_price
#                             exit_price = trade_initial_sl
#                             exit_time = df.iloc[i]["datetime"]
#                             remark = "Initial SL hit"

#                     else:
#                         # Gap Open Outside ISL
                        
#                         in_trade = False
#                         points = df.iloc[i]['open'] - trade_entry_price
#                         exit_price = df.iloc[i]['open']
#                         exit_time = df.iloc[i]["datetime"]
#                         remark = "Gap Open Outside ISL"

#                 elif df.iloc[i]['low'] <= trade_initial_sl:
#                     # Initial SL Hit

#                     in_trade = False
#                     points = trade_initial_sl - trade_entry_price
#                     exit_price = trade_initial_sl
#                     exit_time = df.iloc[i]["datetime"]
#                     remark = "Initial SL Hit"

#                 elif df.iloc[i]['close'] <= trade_final_sl:
#                     # Initial SL Hit

#                     in_trade = False
#                     points = df.iloc[i]['close'] - trade_entry_price
#                     exit_price = df.iloc[i]['close']
#                     exit_time = df.iloc[i]["datetime"]
#                     remark = "TSL Hit"    

#         if points:
#             slippage = SLIPPAGE_ * (entry_price + exit_price)
#             final_points = points - slippage
#             # sl_in_points = abs(entry_price - initial_sl)
#             qty = PORTFOLIO_VALUE * LEVERAGE_ / entry_price
#             trade = {
#                 'Signal Time': signal_time,
#                 'Trade Type': 'LONG',
#                 'Entry Date': entry_time.date(),
#                 'Entry Time': entry_time.time(),
#                 'Entry Price': entry_price,
#                 'Initial SL': trade_initial_sl,
#                 'Final SL': trade_final_sl,
#                 'Exit Date': exit_time.date(),
#                 'Exit Time': exit_time.time(),
#                 'Exit Price': exit_price,
#                 'Points Captured': points,
#                 'After Costs': final_points,
#                 'Remarks': remark,
#                 'Qty': qty,
#                 'ROI%': (points * qty / PORTFOLIO_VALUE) * 100,
#                 'ROI% w cs': (final_points * qty / PORTFOLIO_VALUE) * 100,
#                 'Trade Year': entry_time.year,
#             }
#             # print(trade)
#             tradebook.append(trade)
#             points = 0
#             in_trade = False
#             remark = ''
#             entry_price = None
#             exit_price = None
#             initial_sl = None
#             # stoploss = None
#             is_trailing_active = False

#     return pd.DataFrame(tradebook)

In [375]:
# async def trade(spot_data, index, tf):

#     if index == "bnf":
#         SPREAD = 100
#         index_full = "BANKNIFTY"
#     elif index == "nifty":
#         SPREAD = 50
#         index_full = "NIFTY"

#     start_date = dt.date(2017, 1, 1)
#     end_date = dt.date(2024, 3, 31)

#     flag = False

#     current_date = start_date

#     final_tb = pd.DataFrame()
#     tradebook = []

#     while current_date <= end_date:
#         # print(current_date)

#         if current_date.weekday() >= 5:
#             print("Skipping Weekend ! ", current_date)
#             current_date += dt.timedelta(days=1)
#             continue
#         elif current_date not in list_of_traded_dates:
#             print("Date Avoided ! ", current_date)
#             current_date += dt.timedelta(days=1)
#             continue
#         # elif current_date in list_of_dates_to_avoid_bnf:
#         #     # print('Date Avoided ! ', current_date)
#         #     current_date += dt.timedelta(days=1)
#         #     continue
#         else:
#             print("Working", current_date)
#             spot = spot_data[spot_data["datetime"].dt.date == current_date]
#             spot_open = spot.iloc[0]["open"]
#             # print(spot_open)
#             atm = int(round(spot_open / SPREAD) * SPREAD)
#             # print(atm)

#             current_expiry = await get_expiry(current_date, index)
#             dte = current_expiry - current_date

#             ce_df = await fetch_option_data(
#                 index=index,
#                 start_date=current_date,
#                 end_date=current_date,
#                 start_time=dt.time(9, 15),
#                 end_time=dt.time(15, 30),
#                 expiry=current_expiry,
#                 strike=atm,
#                 asset_class="C",
#             )
#             if not isinstance(ce_df, pl.DataFrame):
#                 print('Data Not Found for Options')
#                 current_date += dt.timedelta(days=1)
#                 continue
#             # print(ce_df.head())
#             ce_df = rename_ohlc_columns(ce_df)
#             # ce_5min = resample(ce_df, "5m")
#             ce_df = resample(ce_df, tf)
#             # call_option = ce_5min.to_pandas()
#             call_option = ce_df.to_pandas()

#             pe_df = await fetch_option_data(
#                 index=index,
#                 start_date=current_date,
#                 end_date=current_date,
#                 start_time=dt.time(9, 15),
#                 end_time=dt.time(15, 30),
#                 expiry=current_expiry,
#                 strike=atm,
#                 asset_class="P",
#             )
#             if not isinstance(pe_df, pl.DataFrame):
#                 print('Data Not Found for Options')
#                 current_date += dt.timedelta(days=1)
#                 continue
#             # print(pe_df.head())
#             pe_df = rename_ohlc_columns(pe_df)
#             # pe_5min = resample(pe_df, "5m")
#             pe_df = resample(pe_df, tf)
#             # put_option = pe_5min.to_pandas()
#             put_option = pe_df.to_pandas()

#             # call_option['datetime'] = pd.to_datetime(call_option['datetime'])
#             # put_option['datetime'] = pd.to_datetime(put_option['datetime'])

#             ce_df = await create_sell_signals(call_option)
#             pe_df = await create_sell_signals(put_option)

#             # print(ce_df.head())
#             # print(pe_df.head())

#             ce_contract = await get_option_contract_name(
#                 symbol=index_full,
#                 strike=atm,
#                 expiry=current_expiry,
#                 opt_type='CE'
#             )

#             pe_contract = await get_option_contract_name(
#                 symbol=index_full,
#                 strike=atm,
#                 expiry=current_expiry,
#                 opt_type='PE'
#             )

#             ce_trades = await option_trading(ce_df, ce_contract, atm, current_expiry)
#             pe_trades = await option_trading(pe_df, pe_contract, atm, current_expiry)

#             final_tb = pd.concat([final_tb, ce_trades, pe_trades], ignore_index=True)

#         current_date += dt.timedelta(days=1)

#     return final_tb

In [376]:
# async def trade_positional(spot_data, index, tf, offset, ma):

#     if index == "bnf":
#         SPREAD = 100
#         index_full = "BANKNIFTY"
#     elif index == "nifty":
#         SPREAD = 50
#         index_full = "NIFTY"

#     start_date = dt.date(2017, 1, 1)
#     end_date = dt.date(2024, 3, 31)

#     flag = False

#     current_date = start_date

#     final_tb = pd.DataFrame()
#     tradebook = []

#     # while current_date <= end_date:
#         # print(current_date)

#     # if current_date.weekday() >= 5:
#     #     print("Skipping Weekend ! ", current_date)
#     #     current_date += dt.timedelta(days=1)
#     #     continue
#     # elif current_date not in list_of_traded_dates:
#     #     print("Date Avoided ! ", current_date)
#     #     current_date += dt.timedelta(days=1)
#     #     continue
#     # # elif current_date in list_of_dates_to_avoid_bnf:
#     # #     # print('Date Avoided ! ', current_date)
#     # #     current_date += dt.timedelta(days=1)
#     # #     continue
#     # else:
#     #     print("Working", current_date)
#     data = resample(spot_data, tf, offset)
#     data_pandas = data.to_pandas()
#     data_pandas = await create_buy_signals(data_pandas)
#     data_pandas = await create_sell_signals(data_pandas)
#     data_pandas = await create_ma(data_pandas, ma)
#     # print(data.head(50))
#     # break
#     tb = await positional(data_pandas)
#     final_tb = pd.concat([tb, final_tb])

#         # current_date += dt.timedelta(days=1)

#     return final_tb

In [377]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2017, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2017, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI% w cs"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI% w cs"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI% w cs"] > 0]["ROI% w cs"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI% w cs"] < 0]["ROI% w cs"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI% w cs"].cumsum() - year_trades["ROI% w cs"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI% w cs"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI% w cs"] > 0]["ROI% w cs"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI% w cs"] < 0]["ROI% w cs"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI% w cs"].cumsum() - combined_df_sorted["ROI% w cs"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation

    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    
    # print(f'{overall_total_roi} , {overall_max_drawdown} , {overall_roi_dd_ratio}')
    
    return {overall_roi_dd_ratio: stats_df8}

In [378]:
stats = generate_stats(tb_pandas, '...')
lol = pd.DataFrame()
for x, y in stats.items():
    lol = pd.DataFrame(y)

lol

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2017,-4.0952,40,37.5000,1.4082,-1.0087,-6.5712,-0.6232,...
2018,-7.4285,50,26.0000,2.2534,-0.9925,-19.0829,-0.3893,...
2019,11.7574,46,36.9565,2.4808,-1.0488,-9.8329,1.1957,...
2020,-1.2571,46,36.9565,1.7252,-1.0547,-19.0040,-0.0662,...
2021,0.3979,44,27.2727,2.5673,-0.9503,-10.2765,0.0387,...
2022,4.9859,49,36.7347,1.9756,-0.9863,-14.4063,0.3461,...
2023,-5.7737,51,41.1765,1.2150,-1.0430,-11.5096,-0.5016,...
2024,-5.4692,37,21.6216,3.3652,-1.1169,-13.1803,-0.4150,...
Overall,-6.8825,363,33.5586,1.8886,-1.0050,-41.0946,-0.1675,...


In [225]:
# tfs = ['15m', '30m', '60m', '1d']
# for i in range(5, 81, 5):
#     for tf in tfs:
#         if tf == '30m' or tf == '60m':
#             ofs = '15m'
#         else:
#             ofs = '0m'
#         variation = f'TF : {tf} , TMA : {i}'
#         print(variation)
#         x = await trade_positional(pl.DataFrame(bnf_1min), 'bnf', tf, ofs, i)
#         x['Entry Datetime'] = x.apply(lambda row: pd.to_datetime(f"{row['Entry Date']} {row['Entry Time']}"), axis=1)
#         x = x.sort_values(by='Entry Datetime')
#         stats = generate_stats(x, variation)
# # x

In [628]:
# x['Entry Datetime'] = x.apply(lambda row: pd.to_datetime(f"{row['Entry Date']} {row['Entry Time']}"), axis=1)
# x = x.sort_values(by='Entry Datetime')

In [54]:
# roi = x['ROI%'].sum()
# roi_w_cs = x['ROI% w cs'].sum()
# roi, roi_w_cs

In [55]:
# x = await trade_positional(pl.DataFrame(bnf_1min), 'bnf', '30m', '15m', 10)
# x['Entry Datetime'] = x.apply(lambda row: pd.to_datetime(f"{row['Entry Date']} {row['Entry Time']}"), axis=1)
# x = x.sort_values(by='Entry Datetime')
# stats = generate_stats(x, variation)

In [56]:
# for a, b in stats.items():
#     print(a)
#     print(b.to_string())

In [57]:
# x.tail()

In [58]:
# x['Slippage in Points'] = x.apply(lambda row: 0.25 / 100 * (row['Entry Price'] + row['Exit Price']), axis=1)

In [59]:
# x['Final Points'] = x.apply(lambda row: row['Points Captured'] - row['Slippage in Points'])
# x['PnL w cs'] = x.apply(lambda row: row['Final Points'] * row['Qty'])
# x['ROI% w cs'] = x.aply(lambda row: row['PnL w cs'] * 100 / 1000000)

In [ ]:
# x